# Create and Train Model

In [2]:
import numpy as np
import config
import os

In [3]:
wav_mfccs_dir = config.WAV_MFCCS
lms_mfccs_dir = config.LMS_MFCCS

wav_mfccs_list = []
wav_labels_list = []

for file_name in os.listdir(wav_mfccs_dir):
    if file_name.endswith('.npy'):
        mfccs = np.load(os.path.join(wav_mfccs_dir, file_name))
        wav_mfccs_list.append(mfccs)
        
        label = file_name.split('_')[0]
        wav_labels_list.append(label)

lms_mfccs_list = []
lms_labels_list = []

for file_name in os.listdir(lms_mfccs_dir):
    if file_name.endswith('.npy'):
        mfccs = np.load(os.path.join(lms_mfccs_dir, file_name))
        lms_mfccs_list.append(mfccs)
        
        label = file_name.split('_')[0]
        lms_labels_list.append(label)



In [4]:
print(wav_mfccs_list[0].shape)
print(lms_mfccs_list[0].shape)


(40, 119)
(40, 775)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


NameError: name 'X' is not defined